# 1. 환경세팅
requirements: conda가 깔려있으며 working directory는 /SpeechGPT/

In [ ]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs
!git lfs install
!git clone https://huggingface.co/fnlp/SpeechGPT-7B-cm
!git clone https://huggingface.co/fnlp/SpeechGPT-7B-com

In [ ]:
!conda create --name SpeechGPT python=3.8
!conda activate SpeechGPT
!conda install pip=24.0
!conda install cudatoolkit=11.7 -c conda-forge
!pip install -r requirements.txt
!pip install -U datasets
# pip install transformers==4.34
!pip install omegaconf==2.3.0
!pip install accelerate==0.26.0
!pip install torch==2.4.1
!pip install torchaudio==2.4.1
!pip install transformers==4.45
!pip install --upgrade huggingface-hub
!pip install bert_score

In [ ]:
!cd utils/speech2unit
!wget https://dl.fbaipublicfiles.com/hubert/mhubert_base_vp_en_es_fr_it3.pt
!wget https://dl.fbaipublicfiles.com/hubert/mhubert_base_vp_en_es_fr_it3_L11_km1000.bin

In [ ]:
!wget https://dl.fbaipublicfiles.com/wavlm/wavlm_base_en_it3_L6_km1000.bin

: 

In [ ]:
!cd utils/vocoder
!wget https://dl.fbaipublicfiles.com/fairseq/speech_to_speech/vocoder/code_hifigan/mhubert_vp_en_es_fr_it3_400k_layer11_km1000_lj/config.json -O config.json
!wget https://dl.fbaipublicfiles.com/fairseq/speech_to_speech/vocoder/code_hifigan/mhubert_vp_en_es_fr_it3_400k_layer11_km1000_lj/g_00500000 -O vocoder.pt

## SpeechGPT CLI inference 테스트

In [ ]:
# /SpeechGPT/ 에서 실행
!python3 speechgpt/src/infer/cli_infer.py --model-name-or-path "./SpeechGPT-7B-cm" --lora-weights "./SpeechGPT-7B-com" --s2u-dir "speechgpt/utils/speech2unit" --vocoder-dir "speechgpt/utils/vocoder" --output-dir "output"

# 2. 데이터 Preprocess

In [ ]:
# speech encoder, kmeans model 잘 동작하는 지 확인
# /SpeechGPT/speechgpt/utils/speech2unit/ 에서 실행
!python3 speech2unit.py --wav "00006.wav"

In [ ]:
# /SpeechGPT/speechgpt/data/ 에서 실행
!mkdir ./stage1
# !python preprocess.py --checkpoint-dir /home/jhwan98/EmoSDS/SpeechGPT/speechgpt/utils/speech2unit/

In [ ]:
# Llama 2 7B 모델 다운로드
# speechgpt/llama/hf/7B 만들고 여기서 다음을 실행
# git clone https://huggingface.co/meta-llama/Llama-2-7b

# Llama 3.2 3B 모델 다운로드
# speechgpt/llama/3_2/3B 만들고 여기서 다음을 실행
git clone https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct

In [ ]:
# /SpeechGPT/speechgpt/ 에서 실행
# bash scripts/ma_pretrain.sh ${NNODE} ${NODE_RANK} ${MASTER_ADDR} ${MASTER_PORT}
# bash scripts/ma_pretrain.sh

In [ ]:
# step 1 style prediction 훈련 데이터 생성
# /SpeechGPT/speechgpt/data 에서 실행
# 실행하기 전에 main() 코드 알맞게 수정할 것
!python3 build_instruct_jsonl.py # step 1
# !python3 merge_jsonl_files.py file1 file2 output_filename # 필요하면 실행
# !python3 merge_jsonl_files.py stage2/emotion_prediction_data_train.jsonl stage2/emotion_prediction_data_eval.jsonl stage2/emotion_prediction_data_train_eval.jsonl
# !python3 merge_jsonl_files.py stage3/dialogue_prediction_data_train.jsonl stage3/dialogue_prediction_data_eval.jsonl stage3/dialogue_prediction_data_train_eval.jsonl

In [ ]:
# step 2 dialogue prediction 훈련 데이터 생성
# /SpeechGPT/speechgpt/data 에서 실행
# !python3 build_instruct_jsonl_step3.py # step 2
!python3 build_instruct_jsonl_asr.py # step 2

# 3. step1: style prediction 훈련 진행

In [ ]:
# step1 학습
# /SpeechGPT/speechgpt/ 에서 실행
!bash scripts/style_sft.sh

# 4. step2: dialogue prediction 훈련 진행

In [ ]:
# step 2 학습
# /SpeechGPT/speechgpt/ 에서 실행
!bash scripts/dialogue_sft.sh
!bash scripts/asr_sft.sh

In [ ]:
# style prediction cli inference
# /SpeechGPT/ 에서 실행
!python3 speechgpt/src/infer/cli_infer_emosds.py --model-name-or-path "./speechgpt/output/stage2/" --s2u-dir "speechgpt/utils/speech2unit" --output-dir "output"
!python3 speechgpt/src/infer/cli_infer_emosds.py --model-name-or-path "./speechgpt/output/stage3/" --s2u-dir "speechgpt/utils/speech2unit" --output-dir "output"

# step 1 ex) "Identify style of given speech. This is input: speechgpt/prompt/school_219_c_1.wav" --> sarcastic EmoSDS: neutral
# step 1 ex) "Identify style of given speech. This is input: speechgpt/prompt/weather_465_c_0.wav" --> sleepy EmoSDS: neutral
# step 1 ex) "Identify style of given speech. This is input: speechgpt/prompt/weather_465_c_2.wav" --> friendly EmoSDS: upbeat

# "The forecast said it might rain later though."
# step 2 ex) "speechgpt/prompt/weather_465_c_0.wav" --> GT: [sleepy][depressed]  Oh man, that's a bummer. I was really looking forward to soaking up the sun all day.
#                                                   --> EmoSDS: [sleepy][neutral]  Yeah, a bit of planning the forecast and we'll play the beginning
#                                                   --> EmoSDS: [bored][friendly]  Well, let's really interesting, but we could have plenty for keeping do (temp:0.7)
#                                                   --> EmoSDS: [bored][neutral]  Well, let's check that. Maybe we should be a strategy to keep our adventure (temp:0.05)
# step 2 ex) "speechgpt/prompt/weather_465_c_2.wav" --> GT: [friendly][friendly]  Hopefully it'll hold off until the evening. Let's pack some umbrellas just in case and keep our fingers crossed!
#                                                   --> EmoSDS: [soft-spoken][friendly]  Let's hold, I'll hold of the long on a big all (temp:0.7)
#                                                   --> EmoSDS: [soft-spoken][friendly]  Let's hold, I can go for our evening, I'm all about it (temp:0.05)


# After unified fine-tuning (15 epochs)

# step 3 ex) "This is input: speechgpt/prompt/weather_465_c_0.wav" --> GT: [sleepy][depressed]  Oh man, that's a bummer. I was really looking forward to soaking up the sun all day.
#                                                                  --> EmoSDS: [neutral] The right products can really make a difference. Answer: [neutral] Absolutely, it\'s all about finding what works for each person. (temp:0.7)
#                                                                  --> EmoSDS: [neutral] The forecast says it might rain later though. Answer: [sad] Oh no, that would ruin our barbecue plans. Let\'s hope the weather holds up. (temp:0.05)
# step 3 ex) "This is input: speechgpt/prompt/weather_465_c_2.wav" --> GT: [friendly][friendly]  Hopefully it'll hold off until the evening. Let's pack some umbrellas just in case and keep our fingers crossed!
#                                                                  --> EmoSDS: [cheerful] The prices seem reasonable for the quality you get. Answer: [cheerful] Totally, it\'s like they\'re giving it away Let\'s go grab the best stuff before it\'s all gone (temp:0.7)
#                                                                  --> EmoSDS: [friendly] They even have a discount for first-time visitors. Answer: [cheerful] Oh, I\'ll mention that to my cousin; she\'s been looking for a new place (temp:0.05)

# After unified fine-tuning (7 epochs)

# step 3 ex) "This is input: speechgpt/prompt/weather_465_c_0.wav" --> GT: [sleepy][depressed]  Oh man, that's a bummer. I was really looking forward to soaking up the sun all day.
#                                                                  --> EmoSDS: [neutral] I think you\'ll look great in it, honestly. Answer: [neutral] Thanks for the advice, I appreciate the help picking it out. (temp:0.7)
#                                                                  --> EmoSDS: [neutral] I\'m thinking of joining the tournament next week. Answer: [neutral] Oh, a tournament sounds interesting; I bet you\'d do great (temp:0.05)
# step 3 ex) "This is input: speechgpt/prompt/weather_465_c_2.wav" --> GT: [friendly][friendly]  Hopefully it'll hold off until the evening. Let's pack some umbrellas just in case and keep our fingers crossed!
#                                                                  --> EmoSDS: [unfriendly] They really need to keep up their defense in the next game though. Answer: [unfriendly] Well, I know, keeping their guard up and staying focused is gonna play a big part in the next game. (temp:0.7)
#                                                                  --> EmoSDS: [friendly] I heard they\'re making a sequel already. Answer: [cheerful] That\'s fast Fingers crossed it\'s just as good, I can\'t wait to see it (temp:0.05)

# After unified fine-tuning (3 epochs)

# step 3 ex) "This is input: speechgpt/prompt/weather_465_c_0.wav" --> GT: [sleepy][depressed]  Oh man, that's a bummer. I was really looking forward to soaking up the sun all day.
#                                                                  --> EmoSDS: [neutral] That sounds like a plan, I\'ll check my schedule. Answer: [cheerful] Great, just let me know whenever you have a moment, and we\'ll make it a movie night (temp:0.7)
#                                                                  --> EmoSDS: [neutral] I wonder how the team will perform next season. Answer: [neutral] Well, if they keep up the good work, I\'m pretty sure they\'ll put up a good fight. (temp:0.05)
# step 3 ex) "This is input: speechgpt/prompt/weather_465_c_2.wav" --> GT: [friendly][friendly]  Hopefully it'll hold off until the evening. Let's pack some umbrellas just in case and keep our fingers crossed!
#                                                                  --> EmoSDS: [neutral] So, what\'s the game plan for tonight? Answer: [neutral] Well, if we play like yesterday, I guess we\'ve got a good chance. (temp:0.7)
#                                                                  --> EmoSDS: [neutral] I wonder how the team will perform next season. Answer: [neutral] Well, if they keep up the good work, I\'m pretty sure they\'ll put up a good fight. (temp:0.05)